In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.plotting import scatter_matrix
from sklearn.metrics import mean_squared_error
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split
from sklearn.externals.six import StringIO  
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error as mse
%matplotlib inline 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score,r2_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.feature_selection import RFECV
import os
from tqdm import tqdm
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'UacLuaenable':                                         'float64', # was 'float32'
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', # was 'float16'
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = [c for c,v in dtypes.items() if v in numerics]
categorical_columns = [c for c,v in dtypes.items() if v not in numerics]
categorical_columns.remove('MachineIdentifier')
# categorical_columns

In [ ]:
train = pd.read_csv(r'../input/train.csv',dtype=dtypes,low_memory=True,nrows=100000)  
print(train.head())
train=reduce_mem_usage(train)

In [ ]:
print(train['AvSigVersion'].unique())
print(type(train['AvSigVersion']))
train['version']=train['AvSigVersion'].str.split('.')
#train['version']=train['AvSigVersion'].apply(lambda x: x.split('.'))
train['version']=[item[0]+(item[1]) for item in train['version']]
train['version']=train['version'].astype(int)
train.groupby('version').sum()


In [ ]:
test = pd.read_csv(r'../input/test.csv',dtype=dtypes,low_memory=True)
test=reduce_mem_usage(test)

In [ ]:
print(test['AvSigVersion'].unique())
print(type(test['AvSigVersion']))
test['version']=test['AvSigVersion'].str.split('.')
test['version']=[item[0]+(item[1]) for item in test['version']]
test['version']=test['version'].astype(int)
test.groupby('version').sum()

In [ ]:
# train.drop('AvSigVersion',axis=1,inplace=True)
# test.drop('AvSigVersion',axis=1,inplace=True)

In [ ]:
def get_percentage_missing(series):
    """ Calculates percentage of NaN values in DataFrame
    :param series: Pandas DataFrame object
    :return: float
    """
    num = series.isnull().sum()
    den = len(series)
    return round(num/den, 2)

# Only include columns that contain any NaN values
df_with_any_null_values = train[train.columns[train.isnull().any()].tolist()]
data =get_percentage_missing(df_with_any_null_values)
filter_df=data[data>0.5]
print(train.shape)
remove_cols=[]
for n,v in filter_df.iteritems():
    remove_cols.append(n)
remove_cols.append('AvSigVersion')

In [ ]:
#Lets start by plotting a heatmap to determine if any variables are correlated
plt.figure(figsize = (12,8))
sns.heatmap(data=train[numerical_columns].corr())
plt.show()
plt.gcf().clear()


In [ ]:
# Create correlation matrix
corr_matrix = train[numerical_columns].corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
for each in to_drop:
    remove_cols.append(each)
remove_cols

In [ ]:
# train.drop('EngineVersion',axis=1,inplace=True)
# train.drop('AppVersion',axis=1,inplace=True)
# train.drop('AvSigVersion',axis=1,inplace=True)
# train.drop('OsVer',axis=1,inplace=True)
# train.drop('MachineIdentifier',axis=1,inplace=True)
# train.drop('Census_OSVersion',axis=1,inplace=True)
train.drop(remove_cols,axis=1,inplace=True)
#train.drop('MachineIdentifier',axis=1,inplace=True)
# #train.dropna(axis=0,how='any')
train.replace(np.nan,0,inplace=True)
categorical_columns=list(set(categorical_columns)-set(remove_cols))

In [ ]:
categorical_columns

In [ ]:
def encode_categorical_columns(x_train, x_test, columns, sort=True):
    train_length = x_train.shape[0]
    for col in tqdm(columns):
        if col == 'MachineIdentifier' or col == 'HasDetections':
            continue
            
        combined_data = pd.concat([x_train[col], x_test[col]])
        combined_data, _ = pd.factorize(combined_data, sort=sort)
        combined_data = pd.Series(combined_data).astype('int32')
        x_train[col] = combined_data.iloc[:train_length].values
        x_test[col] = combined_data.iloc[train_length:].values
        x_train[col] = x_train[col].fillna(0)
        x_test[col] = x_test[col].fillna(0)
        del combined_data
        
    return x_train, x_test
test.drop(remove_cols,axis=1,inplace=True)
test.replace(np.nan,0,inplace=True)
#test.drop('MachineIdentifier',axis=1,inplace=True)
train, test = encode_categorical_columns(train, test, categorical_columns)

In [ ]:
train.drop('MachineIdentifier',axis=1,inplace=True)
x=train.loc[:,train.columns != 'HasDetections']
y=train.loc[:,train.columns == 'HasDetections']
x.head(),y.head()

In [ ]:
test_final=test['MachineIdentifier'].copy()
#train.drop('MachineIdentifier',axis=1,inplace=True)
test.drop('MachineIdentifier',axis=1,inplace=True)

In [ ]:
rf=RandomForestRegressor(random_state=23,verbose=0, warm_start=True,n_jobs=-1,n_estimators=600)
rf.fit(x, y)
imp=pd.DataFrame({'label':x.columns,'imp':rf.feature_importances_})
feature_select=imp[imp['imp']>0.01]['label']
feature_select

In [ ]:
zero_features = list(imp[imp['imp'] == 0.000]['label'])
print('There are %d features with 0.0 importance' % len(zero_features))
print(zero_features)
imp.tail()

In [ ]:
# new_features=feature_select
# test_columns=['SmartScreen','EngineVersion']
# test.replace(np.nan,0,inplace=True)
# test_final=test['MachineIdentifier'].copy()

In [ ]:
test=test[feature_select]
y_train = train[train['version'].astype(int) < 1275 ]['HasDetections']
y_test = train[train['version'].astype(int) >= 1275 ]['HasDetections']

train=train[feature_select]
x_train = train[train['version'].astype(int) < 1275 ]
x_test = train[train['version'].astype(int) >= 1275 ]

x_train.head(),y_test.head(),x_test.head()


del train

### RFE implementation for xgboost

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2),
 'n_estimators':[140,280,560,1020]
}
gsearch1 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1,max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,tree_method='gpu_hist',
 objective= 'binary:logistic', scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch1.fit(x_train,y_train)
# Set the clf to the best combination of parameters
optimal=gsearch1.best_estimator_
print(gsearch1.best_estimator_)
gsearch1.best_params_,gsearch1.best_score_

In [ ]:
param_test2 = {
'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = gsearch1.best_estimator_,
param_grid = param_test2, scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch2.fit(x_train,y_train)
print(gsearch2.best_estimator_)
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
#score improved from 0.598 to 0.600
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = gsearch2.best_estimator_,
param_grid = param_test3, scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch3.fit(x_train,y_train)
print(gsearch3.best_estimator_)
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
#subsample and colsample both comes as 0.8. Let try with 0.05 diff
param_test4 = {
 'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch4 = GridSearchCV(estimator = gsearch3.best_estimator_,
param_grid = param_test4, scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch4.fit(x_train,y_train)
print(gsearch4.best_estimator_)
gsearch4.best_params_, gsearch4.best_score_
##no diff found, so will use the previous parameter

In [ ]:
param_test5 = {
 'reg_alpha':[1e-5, 1e-2, 0.1,0, 0.001, 0.005, 0.01, 0.05]
}
gsearch5 = GridSearchCV(estimator = gsearch3.best_estimator_,
param_grid = param_test5, scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch5.fit(x_train,y_train)
print(gsearch5.best_estimator_)
gsearch5.best_params_, gsearch5.best_score_

In [ ]:
xgb_final = gsearch5.best_estimator_
xgb_final.fit(x_train,y_train)


In [ ]:
param_test6 = {
 'learning_rate':np.linspace(0,1,30)
}
gsearch6 = GridSearchCV(estimator = gsearch5.best_estimator_,
param_grid = param_test6, scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch6.fit(x_train,y_train)
print(gsearch6.best_estimator_)
gsearch6.best_params_, gsearch6.best_score_


In [ ]:
# xgb_final=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#        colsample_bytree=0.8, gamma=0.0, learning_rate=0.1379,
#        max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
#        n_estimators=1020, n_jobs=-1, scoring = 'roc_auc',
#        objective='binary:logistic', random_state=0, reg_alpha=0,
#        reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
#        subsample=0.9, tree_method='gpu_hist')

In [ ]:
xgb_final = gsearch6.best_estimator_
#df_trainC['version']=df_trainC['version'].values.astype(int)
result=xgb_final.fit(x_train,y_train)
#df_trainD['version']=df_trainD['version'].values.astype(int)
preds=xgb_final.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
# print(test.columns)
#test['version']=test['version'].values.astype(int)

ypred_final=xgb_final.predict(test) 
# print(ypred_final.shape)
# print(test_final.shape)
output_xgb=pd.DataFrame({'MachineIdentifier':test_final,'HasDetections':ypred_final})
output_xgb.to_csv(r'second_file.csv',index=False)

In [ ]:
ITERATIONS = 4
bayes_cv_tuner = BayesSearchCV(
    estimator = xgb.XGBRegressor(
        n_jobs = -1,
        tree_method='gpu_hist',
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        #n_estimators=1020
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (0, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'min_child_weight': (0, 5),
        'n_estimators': [600,1020,1500],
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = -1,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__


In [ ]:
bayes_cv_tuner.fit(x_train,y_train, callback=status_print)
#bayes_cv_tuner.fit(df_trainC.loc[:,df_trainC.columns != ['HasDetections']], df_trainC['HasDetections'], callback=status_print)

In [ ]:
xgb_cv=XGBRegressor(colsample_bylevel= bayes_cv_tuner.best_params_['colsample_bylevel'],
                    colsample_bytree= bayes_cv_tuner.best_params_['colsample_bytree'], 
                    gamma= bayes_cv_tuner.best_params_['gamma'],
                    learning_rate= bayes_cv_tuner.best_params_['learning_rate'], 
                    max_delta_step= bayes_cv_tuner.best_params_['max_delta_step'],
                    max_depth= bayes_cv_tuner.best_params_['max_depth'],
                    min_child_weight= bayes_cv_tuner.best_params_['max_depth'], 
                    reg_alpha= bayes_cv_tuner.best_params_['reg_alpha'], 
                    reg_lambda= bayes_cv_tuner.best_params_['reg_lambda'], 
                    scale_pos_weight= bayes_cv_tuner.best_params_['scale_pos_weight'],
                    subsample= bayes_cv_tuner.best_params_['subsample'],
                   tree_method='gpu_hist')
xgb_cv.fit(x_train,y_train)

In [ ]:
preds=xgb_cv.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
y_test = xgb_cv.predict(test)
output_xgb=pd.DataFrame({'MachineIdentifier':test_final,'HasDetections':y_test})
output_xgb.to_csv(r'first_file.csv',index=False)